In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("raw_predictors_until2013.csv")
sLength = len(data['yyyymm'])
# create new columns
for x in range(1,25):
    t = str(x)
    data['Index - '+t] = np.nan 
print(np.random.randn(sLength))

# add correct values to columns
for z in range(1,25):
    l = []
    t = str(z)
    for y in range(24,sLength):
        hold = data['Index'][y-z]
        l.append(hold)
    data['Index - '+t][24:] = l
print(data)

[ 1.85042898 -1.30203212 -0.09808326 ... -0.99946244 -0.30188952
 -1.45009041]


/afs/crc.nd.edu/user/p/pbui/pub/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/afs/crc.nd.edu/user/p/pbui/pub/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


      yyyymm     Index      D12       E12       b/m     tbl     AAA     BAA  \
0     192701     13.21   0.6967    1.2290  0.443706  0.0323  0.0466  0.0561   
1     192702     13.84   0.7033    1.2180  0.428501  0.0329  0.0467  0.0559   
2     192703     13.93   0.7100    1.2080  0.469765  0.0320  0.0462  0.0554   
3     192704     14.17   0.7167    1.1970  0.456754  0.0339  0.0458  0.0548   
4     192705     14.91   0.7233    1.1860  0.434783  0.0333  0.0457  0.0550   
5     192706     14.77   0.7300    1.1750  0.452385  0.0307  0.0458  0.0555   
6     192707     15.73   0.7367    1.1640  0.414553  0.0296  0.0460  0.0555   
7     192708     16.43   0.7433    1.1530  0.396227  0.0270  0.0456  0.0548   
8     192709     17.14   0.7500    1.1430  0.380586  0.0268  0.0454  0.0542   
9     192710     16.23   0.7567    1.1320  0.413801  0.0308  0.0451  0.0538   
10    192711     17.33   0.7633    1.1210  0.379396  0.0304  0.0449  0.0535   
11    192712     17.66   0.7700    1.1100  0.374689 

13.21


In [6]:
sLength = len(data['yyyymm'])
# add the bullMarket column with random ints (to be replaced in next cell)
data['bull'] = pd.Series(np.random.randn(sLength), index=data.index)
# Create bullMarkert for ret_sp500+1
bull = []
for data in data['CRSP_SPvw']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
data['bull'] = bull

data.shape

TypeError: 'float' object is not subscriptable

In [3]:

# Rename columns
sp500.columns = ['Date', 'sp500', 'sp500+1','ret_sp500', 'ret_sp500+1']
sp500 = sp500.dropna()
sLength = len(sp500['Date'])
# add the bullMarket column with random ints (to be replaced in next cell)
sp500['bullMarket'] = pd.Series(np.random.randn(sLength), index=sp500.index)

# Create bullMarkert for ret_sp500+1
bull = []
for data in sp500['ret_sp500+1']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
sp500['bullMarket'] = bull
# print(sp500)

sp500.shape
# print(sp500)

(14275, 6)

In [27]:
tdata = pd.read_csv("Total_Data_no_dots.csv",low_memory=False)

result = pd.merge(sp500, tdata, on='Date', how='left')

result  = result.dropna()

# assume we want to predict the raw index of sp500 at T+1, given all the info at T

y_reg = result['ret_sp500+1']

y_cal = result['bullMarket']
 
# list(result)

result = result.drop(columns=['VOX Close', 'DGS1', 'DGS10', 'T10Y2Y', 'DBAA', 'USD1MTD156N', 'DCOILBRENTEU', 'DTWEXB', 'DEXUSEU', 'T10YIE', 'T5YIFR', 'BAMLH0A0HYM2EY'])

x = result.drop(columns=['Date','sp500_x','sp500+1','ret_sp500+1','Unnamed: 0'])

# print(result.dtypes)

In [29]:
# Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import KFold

kf = KFold(x.shape[0], n_folds=2, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y_reg.iloc[train_index], y_reg.iloc[test_index]
    y_train_cal, y_test_cal = y_cal.iloc[train_index], y_cal.iloc[test_index]

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)

    y_pred = regr.predict(x_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))
    
print( "the main take way: the mean squared error is always within 1% ")


MSE:  0.8108592714839336
MSE:  0.8220161493992437
the main take way: the mean squared error is always within 1% 


In [31]:
# clssification 

# decision tree CART algorithm

from sklearn import tree
from sklearn.metrics import confusion_matrix
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train_cal)
predicted = clf.predict(x_test)
# print (predicted)
c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: 'perfect' prediction, overfitting problem")


[[2132    0]
 [   0 2242]]
precision:  1.0
recall:  1.0
the main take way: 'perfect' prediction, overfitting problem


In [26]:
# naive Bayes 
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(x_train, y_train_cal).predict(x_test)

predicted= gnb.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is slightly better than a coin flip")

[[ 120  143]
 [2012 2099]]
precision:  0.9362176628010704
recall:  0.5105813670639747


In [34]:
# Support Vector Classification

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x_train, y_train_cal) 

predicted= clf.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is way more better than naive Bayes ")

[[1063    0]
 [1069 2242]]
precision:  1.0
recall:  0.6771368166717004
the main take way: High Precision, recall is way more better than naive Bayes 
